In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
corpus = [
    "I like to eat healthy food.",
    "Food is essential for a healthy life.",
    "I prefer to eat healthy meals."
]
vectorizer = TfidfVectorizer(stop_words='english')  # Remove common stop words
X = vectorizer.fit_transform(corpus)
print("TF-IDF matrix")
print(X)
print(type(X))

result = X.toarray()
print(result)

TF-IDF matrix
  (0, 5)	0.6317450542765208
  (0, 0)	0.4804583972923858
  (0, 3)	0.3731188059313277
  (0, 2)	0.4804583972923858
  (1, 3)	0.34520501686496574
  (1, 2)	0.444514311537431
  (1, 1)	0.5844829010200651
  (1, 4)	0.5844829010200651
  (2, 0)	0.444514311537431
  (2, 3)	0.34520501686496574
  (2, 7)	0.5844829010200651
  (2, 6)	0.5844829010200651
<class 'scipy.sparse._csr.csr_matrix'>
[[0.4804584  0.         0.4804584  0.37311881 0.         0.63174505
  0.         0.        ]
 [0.         0.5844829  0.44451431 0.34520502 0.5844829  0.
  0.         0.        ]
 [0.44451431 0.         0.         0.34520502 0.         0.
  0.5844829  0.5844829 ]]


In [10]:
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=2)
x_reduced =svd.fit_transform(X)
print(x_reduced)

[[ 0.82414414  0.        ]
 [ 0.65886672  0.66363902]
 [ 0.65886672 -0.66363902]]


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
def data_preprocessing(X,y):
    vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
    X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
    X_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    
    base_model = LogisticRegression(multi_class='multinomial',solver='lbfgs',max_iter=1000,C=1.0)
    multi_model = MultiOutputClassifier(base_model)
    multi_model.fit(X_tfidf,y_train)
    y_pred = multi_model.predict(X_test_tfidf)
    print("Accuracy is: ",accuracy_score(y_test,y_pred))
    
data_preprocessing(X,y)


In [ ]:
from sklearn.neural_network import  MLPClassifier

mlp_classifier = MLPClassifier(
    hidden_layer_sizes=(3,),#(128, 64, 32),  # Three hidden layers
    activation='tanh',                 # ReLU activation function
    solver='sgd',                     # Adam optimizer
    alpha=0.0001,                      # L2 regularization parameter
    batch_size='auto',                 # Automatic batch size
    learning_rate='adaptive',          # Adaptive learning rate
    max_iter=1000,                     # Maximum number of iterations
    random_state=42,                   # For reproducibility
    verbose=True                       # Print progress
)

mlp_classifier.fit(X_tfidf,y_train)
y_pred = mlp_classifier.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter

# Sample corpus
corpus = [
    "I love natural language processing",
    "Language models are fascinating",
    "Natural language processing is fun"
]

# Preprocessing: Tokenize and build vocabulary
tokens = [sentence.lower().split() for sentence in corpus]
vocab = set(word for sentence in tokens for word in sentence)
vocab = list(vocab)

# Create co-occurrence matrix
co_occurrence = np.zeros((len(vocab), len(vocab)))
window_size = 2

for sentence in tokens:
    for i, word in enumerate(sentence):
        for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
            if i != j:
                co_occurrence[vocab.index(word), vocab.index(sentence[j])] += 1

print(co_occurrence.shape,len(vocab))

# Convert to probabilities
total_count = np.sum(co_occurrence)
p_w_c = co_occurrence / total_count
p_w = np.sum(p_w_c, axis=1)
p_c = np.sum(p_w_c, axis=0)

# Compute PMI
pmi = np.log2(p_w_c / (p_w[:, None] * p_c[None, :]) + 1e-10)  # Add small value to avoid division by zero

# Apply PPMI
ppmi = np.maximum(pmi, 0)

# Convert to DataFrame for better readability
ppmi_df = pd.DataFrame(ppmi, index=vocab, columns=vocab)

print("PPMI Matrix:")
print(ppmi_df)


array([5, 0, 3, 2, 3, 2, 1, 4, 0, 3, 7, 6])